## Installation

In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

e:\Kuliah\Kuliah\Semester 6\Riset\crowd-detection-tult\yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.
C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import

In [10]:
import torch
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [11]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-5-1 Python-3.12.5 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [12]:
cap = cv2.VideoCapture('E:/Kuliah/Kuliah/Semester 6/Riset/crowd-detection-tult/video/test.mp4')

In [14]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Deteksi objek
    results = model(frame)
    results.render()
    frame = results.ims[0].copy()


    # Filter hanya person
    df = results.pandas().xyxy[0]
    df = df[df['name'] == 'person']

    # Inisialisasi zona
    h, w = frame.shape[:2]
    zones = {
        'Kiri Atas': [(0, 0), (w//2, 0), (w//2, h//2), (0, h//2)],
        'Kanan Atas': [(w//2, 0), (w, 0), (w, h//2), (w//2, h//2)],
        'Kiri Bawah': [(0, h//2), (w//2, h//2), (w//2, h), (0, h)],
        'Kanan Bawah': [(w//2, h//2), (w, h//2), (w, h), (w//2, h)]
    }

    zone_counts = {z: 0 for z in zones}

    for _, row in df.iterrows():
        x_center = int((row['xmin'] + row['xmax']) / 2)
        y_center = int((row['ymin'] + row['ymax']) / 2)

        for zone_name, polygon in zones.items():
            if cv2.pointPolygonTest(np.array(polygon, np.int32), (x_center, y_center), False) >= 0:
                zone_counts[zone_name] += 1
                break

    # Gambar zona dan teks jumlah orang
    for zone_name, polygon in zones.items():
        cv2.polylines(frame, [np.array(polygon, np.int32)], isClosed=True, color=(0, 255, 0), thickness=2)
        x_text, y_text = polygon[0]
        cv2.putText(frame, f"{zone_name}: {zone_counts[zone_name]}", (x_text + 10, y_text + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
    cv2.imshow("Deteksi Per Wilayah", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master\models\comm

In [15]:
cap.release()
cv2.destroyAllWindows()